In [2]:
import findspark
findspark.init()

In [3]:
import pyspark

In [4]:
conf = pyspark.SparkConf().setAppName("Guide")
sc = pyspark.SparkContext(conf=conf)

###### BASIC

In [5]:
lines = sc.textFile("in.txt")
lineLengths = lines.map(lambda s: len(s))
# lineLengths.persist() to keep lineLengths in cache -> the difference between RDD and Map&Reduce
totalLength = lineLengths.reduce(lambda a, b: a + b)
print(totalLength)

141


###### Passing Function

In [ ]:
class MyClass(object):
    def __init__(self):
        self.field = "Hello"
    def doStuff(self, rdd):
        # copy field into local variable in order to avoid send the whole class object but just field
        field = self.field
        return rdd.map(lambda s: field + s)

###### Understanding Closure

In [ ]:
counter = 0
rdd = sc.parallelize(data)

# Wrong: Don't do this!!
def increment_counter(x):
    # still local counter on each executor due to closure
    global counter
    counter += x
rdd.foreach(increment_counter)

print("Counter value: ", counter)

###### Transformation

| Transformation(params) | Meaning |
|-----|-----|
| map(func) | literal meaning |
| filter(func) | literal meaning |
| flatMap(func) | map + flatten |
| mapPartitions(func of generator) | map in partitions |
| mapPartitionsWithIndex(func of generator) | map in partitions with corresponding index | 
| sample(withReplacement, fration, seed) | literal meaning |
| union(dataset) | literal meaning |
| intersection(dataset) | literal meaning |
| distinct | return a new dataset with unique element |
| groupByKey | literal meaning |
| reduceByKey(func) | literal meaning |
| aggregateByKey(seqOP, combOP) | aggregate based on self-defined function |
| sortByKey | literal meaning |
| join(dataset) | literal meaning |
| cogroup(dataset) | group within two dataset |
| cartesian(dataset | cartesian product |
| pipe(command) | execute bash script |
| coalece(number) | reduce number |
| repartition(number) | reshuffle the data |
| repartitionAndSortWithPartition

###### Action

| Action | Meaning |
| ----- | ----- |
| reduce(func) | literal meaning | 
| collect | return all element |
| count | literal meaning |
| first | literal meaning |
| take(n) | first()==take(1) |
| takeSample(withReplacement, number, seed) | literal meaning |
| takeOrdered | take with a custom comparator |
| saveAsTextFile(path) | literal meaning |
| saveAsSequenceFile(path) | literal meaning |
| saveAsObjectFile(path) | literal meaning |
| countByKey() | literal meaning |
| foreach() | literal meaning |

###### Shuffle

###### RDD Persisence

###### Shared Variables

* boardcast -> read only variable
* accumulators -> added only variable -> activated by action thus each part only calculated once

###### Demo

In [ ]:
# demo1->wordcount
text_file = sc.textFile("hdfs://...")
counts = text_file.flatMap(lambda line: line.split(" ")) \
             .map(lambda word: (word, 1)) \
             .reduceByKey(lambda a, b: a + b)
counts.saveAsTextFile("hdfs://...")

In [ ]:
# demo2->estimate Pi
def inside(p):
    x, y = random.random(), random.random()
    return x*x + y*y < 1

count = sc.parallelize(xrange(0, NUM_SAMPLES)) \
             .filter(inside).count()
print "Pi is roughly %f" % (4.0 * count / NUM_SAMPLES)

###### Get hands dirty
* Task: Write a Spark program to find the top 10 products based on the number of user reviews.
* Dataset: Use the Musical Instruments review file (5-core) and metadata from the Amazon product dataset (http://jmcauley.ucsd.edu/data/amazon/links.html).

In [6]:
sqlcontext = pyspark.SQLContext(sc)

In [7]:
# unique reviewerID for each product
review = sqlcontext.read.schema("asin String, reviewerID String").json ("./Musical_Instruments_5.json").select(["asin","reviewerID"])
asin_ids = review.rdd.map(lambda x: (x.asin,x.reviewerID)).distinct()
asin_ids = asin_ids.map(lambda x: (x[0],1)).reduceByKey(lambda a,b:a+b)

In [8]:
asin_ids.sortBy(lambda x:-x[1]).take(10)

[('B003VWJ2K8', 163),
 ('B0002E1G5C', 143),
 ('B0002F7K7Y', 116),
 ('B003VWKPHC', 114),
 ('B0002H0A3S', 93),
 ('B0002CZVXM', 74),
 ('B0006NDF8A', 71),
 ('B0009G1E0K', 69),
 ('B0002E2KPC', 68),
 ('B0002GLDQM', 67)]

In [11]:
# product-price pair
product = sqlcontext.read.json("./meta_Musical_Instruments.json")
product_price = product.rdd.map(lambda x:(x.asin,x.price))
product_price = product_price.groupByKey().map(lambda x: (x[0],list(set(x[1]))))

In [12]:
product_price.join(asin_ids).sortBy(lambda x: -x[1][1]).take(10)

[('B0002E1G5C', ([6.92], 143)),
 ('B0002F7K7Y', ([1.96], 116)),
 ('B0002H0A3S', ([0.2], 93)),
 ('B0002CZVXM', ([10.22], 74)),
 ('B0006NDF8A', ([10.99], 71)),
 ('B0009G1E0K', ([6.46], 69)),
 ('B0002E2KPC', ([9.92], 68)),
 ('B0002GLDQM', ([1.54], 67)),
 ('B004XNK7AI', ([9.95], 65)),
 ('B005FKF1PY', ([12.56], 63))]

The top10 products based on the number of unique reviewers. 
However, bug remains in reading json which ignore several records. 

In [13]:
sc.stop()